# CellLine

lamindb provides access to the following public protein ontologies through [lnschema-bionty](https://github.com/laminlabs/lnschema-bionty):

1. [Cell Line Ontology](https://github.com/CLO-ontology/CLO)

Here we show how to access and search CellLine ontologies to standardize new data.

## Setup

In [ ]:
!lamin init --storage ./test-cellline --schema bionty

In [ ]:
import lnschema_bionty as lb
import pandas as pd

## PublicOntology objects

Let us create a public ontology accessor with {meth}`~docs:lnschema_bionty.dev.BioRegistry.public`, which chooses a default public ontology source from {class}`~docs:lnschema_bionty.PublicSource`.
It's a [PublicOntology](https://lamin.ai/docs/lnschema_bionty.dev.publicontology) object, which you can think about as a public registry:

In [ ]:
celllines = lb.CellLine.public(organism="all")
celllines

As for registries, you can export the ontology as a `DataFrame`:

In [ ]:
df = celllines.df()
df.head()

Unlike registries, you can also export it as a Pronto object via `public.ontology`.

## Look up terms

As for registries, terms can be looked up with auto-complete:

In [ ]:
lookup = celllines.lookup()

The `.` accessor provides normalized terms (lower case, only contains alphanumeric characters and underscores):

In [ ]:
lookup.hek293

To look up the exact original strings, convert the lookup object to dict and use the `[]` accessor:

In [ ]:
lookup_dict = lookup.dict()
lookup_dict["HEK293"]

By default, the `name` field is used to generate lookup keys. You can specify another field to look up:

In [ ]:
lookup = celllines.lookup(celllines.ontology_id)

In [ ]:
lookup.clo_0000469

## Search terms

Search behaves in the same way as it does for registries:

In [ ]:
celllines.search("hek293").head(3)

By default, search also covers synonyms:

In [ ]:
celllines.search("Human Embryonic Kidney 293").head(3)

You can turn this off synonym by passing `synonyms_field=None`:

In [ ]:
celllines.search("Human Embryonic Kidney 293", synonyms_field=None).head(3)

Search another field (default is `.name`):

In [ ]:
celllines.search(
    "suspension cell line",
    field=celllines.definition,
).head()

## Standardize CellLine identifiers

Let us generate a `DataFrame` that stores a number of CellLine identifiers, some of which corrupted:

In [ ]:
df_orig = pd.DataFrame(
    index=[
        "253D cell"
        "HEK293"
        "2C1H7 cell"
        "283TAg cell"
        "This cellline does not exist",
    ]
)
df_orig

We can check whether any of our values are validated against the ontology reference:

In [ ]:
validated = celllines.validate(df_orig.index, celllines.name)
df_orig.index[~validated]

## Ontology source versions

For any given entity, we can choose from a number of versions:

In [ ]:
lb.PublicSource.filter(entity="CellLine").df()

When instantiating a Bionty object, we can choose a source or version:

In [ ]:
public_source = lb.PublicSource.filter(
    source="clo", version="2022-03-21", organism="all"
).one()
celllines= lb.CellLine.public(public_source=public_source)
celllines

The currently used ontologies can be displayed using:

In [ ]:
lb.PublicSource.filter(currently_used=True).df()

In [ ]:
!lamin delete --force test-cellline
!rm -r test-cellline